### Ingestion de la carpeta "movie_language"

In [0]:
dbutils.widgets.text('p_enviroment', '')
v_enviroment = dbutils.widgets.get('p_enviroment')

In [0]:
dbutils.widgets.text('p_file_date', '2024-12-30')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer el archivo CSV usando DataFrameReader de Spark

In [0]:
from pyspark.sql.types import *

In [0]:
movies_languages_schema = StructType(fields=[
    StructField('movieId', IntegerType(), True),
    StructField('languageId', IntegerType(), True),
    StructField('languageRoleId', IntegerType(), True)
])

In [0]:
movies_languages_df = spark.read \
    .schema(movies_languages_schema) \
    .option('multiline', True) \
    .json(f'{bronze_folder_path}/{v_file_date}/movie_language')

### Paso 2 - Renombrar, eliminar y Agregar Columnas
1. "movieId" renombrar a "movie_id"
2. "languageId" renombrar a "language_id"
3. Eliminar columna "languageRoleId"
4. Agregar columna "ingestion_date"
5. Agregar columna "enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit, col

In [0]:
movies_languages_with_column_df = add_ingestion_date(movies_languages_df) \
    .withColumnRenamed('movieId', 'movie_id') \
    .withColumnRenamed('languageId', 'language_id') \
    .withColumn('enviroment', lit('Production')) \
    .withColumn('file_date', lit(v_file_date))

In [0]:
movies_languages_final_df = movies_languages_with_column_df.drop(col('languageRoleId'))

### Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(movies_languages_final_df, 'movie_silver','movies_languages', 'file_date')

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.language_id = src.language_id AND tgt.file_date  = src.file_date'
merge_delta_lake(movies_languages_final_df, "movie_silver", "movies_languages", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.movies_languages
GROUP BY file_date;

In [0]:
display(spark.read.format('delta').load(f'{silver_folder_path}/movies_languages'))

In [0]:
dbutils.notebook.exit("Success")